- ageグループの切り方を[0, 5, 13, 18, 60, 100]に変えた
- 出費をtwoとthreeのみ使い、特徴量を最小限に

In [16]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import catboost as cb

import optuna
from sklearn.model_selection import StratifiedGroupKFold
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# Load datasets
train = pd.read_csv("../dataset/train.csv")
test = pd.read_csv("../dataset/test.csv")

def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """欠損値を補完せずに特徴量を生成する"""
    data = df.copy()

    # Cabin情報を分解（欠損はNaNのまま保持）
    if 'Cabin' in data.columns:
        cabin_split = data['Cabin'].str.split('/', expand=True)
        data['Deck'] = cabin_split[0]
        data['Num'] = pd.to_numeric(cabin_split[1], errors='coerce')  # NaNそのまま
        data['Side'] = cabin_split[2]
    else:
        data['Deck'] = np.nan
        data['Num'] = np.nan
        data['Side'] = np.nan

    # 支出関連特徴量（NaNはそのまま残す）
    expense_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    two_expense = ['FoodCourt', 'ShoppingMall']
    three_expense = ['RoomService', 'Spa', 'VRDeck']
    for col in expense_cols:
        if col not in data.columns:
            data[col] = np.nan

    # 総支出額（NaNのまま計算 → NaNを含む場合はNaN）
    data['TotalExpense'] = data[expense_cols].sum(axis=1, skipna=False)
    data['TwoExpense'] = data[two_expense].sum(axis=1, skipna=False)
    data['ThreeExpense'] = data[three_expense].sum(axis=1, skipna=False)
    data['HasExpense'] = data[expense_cols].gt(0).any(axis=1).astype(int)


    # 年齢グループ（NaNは'Unknown'にせず、NaNのまま）
    bins = [0, 5, 13, 18, 60, 100]
    labels = ['Child', 'Teen', 'Young', 'Adult', 'Senior']
    data['AgeGroup'] = pd.cut(data['Age'], bins=bins, labels=labels, right=False)

    # Name情報から家族サイズを生成（欠損はNaN）
    if 'Name' in data.columns:
        data['FamilyName'] = data['Name'].str.split().str[-1]
        family_counts = data['FamilyName'].value_counts()
        data['FamilySize'] = data['FamilyName'].map(family_counts)
        data['IsAlone'] = (data['FamilySize'] == 1).astype(int)
    else:
        data['FamilyName'] = np.nan
        data['FamilySize'] = np.nan
        data['IsAlone'] = np.nan

    # PassengerId情報（欠損時はNaN）
    if 'PassengerId' in data.columns:
        passenger_split = data['PassengerId'].str.split('_', expand=True)
        data['PassengerGroup'] = passenger_split[0]
        data['PassengerNum'] = pd.to_numeric(passenger_split[1], errors='coerce')
    else:
        data['PassengerGroup'] = np.nan
        data['PassengerNum'] = np.nan

    return data



# =========================================================
# 2. 特徴量生成
# =========================================================
train_features = create_features(train)
test_features = create_features(test)

feature_cols = [
    'HomePlanet', 'CryoSleep', 'Destination',  'VIP',
    'Side', 'Num',
    'TwoExpense', 'ThreeExpense', 'AgeGroup'
]
# 'Age', 'Deck', 'FamilySize', 'IsAlone', 'HasExpense','TotalExpense', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',

X_full = train_features[feature_cols].copy()
X_test_full = test_features[feature_cols].copy()
y_full = train['Transported']

# カテゴリ列をcategory型に変換（NaNも保持）
categorical_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'Side', 'AgeGroup'
]

for col in categorical_features:
    X_full[col] = X_full[col].astype('category')
    X_test_full[col] = X_test_full[col].astype('category')

print(f"使用特徴量数: {len(feature_cols)}")
print(f"特徴量一覧: {feature_cols}")
print(f"データシェイプ: Train {X_full.shape}, Test {X_test_full.shape}")

使用特徴量数: 9
特徴量一覧: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Side', 'Num', 'TwoExpense', 'ThreeExpense', 'AgeGroup']
データシェイプ: Train (8693, 9), Test (4277, 9)


## lightgbm max leaves4許容

In [5]:
def objective(trial: optuna.Trial) -> float:
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'n_estimators': 600,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 15, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 4),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 1.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 1.0, log=True),
        'random_state': 42,
        'metric': 'None',  # metricはfitで与える
    }

    # --- 重要: 同じPassengerGroupの人を同じsplitに入れる ---
    # create_featuresで作った PassengerGroup列を使う
    groups = train_features['PassengerGroup'].astype(str)

    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    fold_scores = []

    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_full, y_full, groups=groups), start=1):
        X_trn, X_val = X_full.iloc[trn_idx], X_full.iloc[val_idx]
        y_trn, y_val = y_full.iloc[trn_idx], y_full.iloc[val_idx]

        model = lgb.LGBMClassifier(**params)
        callbacks = [
            lgb.early_stopping(stopping_rounds=50, verbose=False),
            lgb.log_evaluation(period=0),
        ]

        model.fit(
            X_trn, y_trn,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',  # LightGBM標準指標
            categorical_feature=categorical_features,
            callbacks=callbacks,
        )

        y_pred = model.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        fold_scores.append(acc)

        trial.report(float(np.mean(fold_scores)), step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return float(np.mean(fold_scores))

# --- Optuna設定 ---
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=2)
study = optuna.create_study(direction='maximize', sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\n=== Optuna 結果 ===")
print(f"Best CV accuracy: {study.best_value:.5f}")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

# --- 最終モデル学習 ---
best_params = study.best_params | {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'n_estimators': 600,
    'metric': 'None',
    'random_state': 42,
}
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X_full, y_full, categorical_feature=categorical_features)
print("\n最終モデル学習完了")


[I 2025-10-20 00:13:24,735] A new study created in memory with name: no-name-fede4fcd-3e18-4b40-ab0f-d06837e022af
Best trial: 0. Best value: 0.807878:   2%|▎         | 1/40 [00:03<02:35,  3.98s/it]

[I 2025-10-20 00:13:28,727] Trial 0 finished with value: 0.80787830440427 and parameters: {'learning_rate': 0.02757359293934948, 'num_leaves': 49, 'max_depth': 4, 'min_data_in_leaf': 64, 'feature_fraction': 0.7468055921327309, 'bagging_fraction': 0.7467983561008608, 'bagging_freq': 1, 'lambda_l1': 0.39676050770529864, 'lambda_l2': 0.06358358856676251}. Best is trial 0 with value: 0.80787830440427.


Best trial: 1. Best value: 0.809038:   5%|▌         | 2/40 [00:06<01:50,  2.91s/it]

[I 2025-10-20 00:13:30,891] Trial 1 finished with value: 0.8090382785332931 and parameters: {'learning_rate': 0.06803900745073703, 'num_leaves': 15, 'max_depth': 4, 'min_data_in_leaf': 85, 'feature_fraction': 0.7637017332034828, 'bagging_fraction': 0.7545474901621302, 'bagging_freq': 1, 'lambda_l1': 0.008179499475211672, 'lambda_l2': 0.03752055855124281}. Best is trial 1 with value: 0.8090382785332931.


Best trial: 1. Best value: 0.809038:   8%|▊         | 3/40 [00:09<02:00,  3.25s/it]

...

[I 2025-10-20 00:14:42,104] Trial 37 pruned. 


Best trial: 33. Best value: 0.809873:  98%|█████████▊| 39/40 [01:18<00:01,  1.97s/it]

[I 2025-10-20 00:14:43,656] Trial 38 pruned. 


Best trial: 33. Best value: 0.809873: 100%|██████████| 40/40 [01:20<00:00,  2.01s/it]


[I 2025-10-20 00:14:44,982] Trial 39 pruned. 

=== Optuna 結果 ===
Best CV accuracy: 0.80987
learning_rate: 0.03640470860180394
num_leaves: 24
max_depth: 4
min_data_in_leaf: 58
feature_fraction: 0.7531453327915494
bagging_fraction: 0.8011825624412329
bagging_freq: 1
lambda_l1: 0.9963418180924837
lambda_l2: 0.17245269987261233

最終モデル学習完了


## lightgbm max leaves3

In [17]:
def objective(trial: optuna.Trial) -> float:
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'n_estimators': 600,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 15, 50),
        'max_depth': 3,
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-3, 1.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-3, 1.0, log=True),
        'random_state': 42,
        'metric': 'None',  # metricはfitで与える
    }

    # --- 重要: 同じPassengerGroupの人を同じsplitに入れる ---
    # create_featuresで作った PassengerGroup列を使う
    groups = train_features['PassengerGroup'].astype(str)

    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    fold_scores = []

    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_full, y_full, groups=groups), start=1):
        X_trn, X_val = X_full.iloc[trn_idx], X_full.iloc[val_idx]
        y_trn, y_val = y_full.iloc[trn_idx], y_full.iloc[val_idx]

        model = lgb.LGBMClassifier(**params)
        callbacks = [
            lgb.early_stopping(stopping_rounds=50, verbose=False),
            lgb.log_evaluation(period=0),
        ]

        model.fit(
            X_trn, y_trn,
            eval_set=[(X_val, y_val)],
            eval_metric='binary_logloss',  # LightGBM標準指標
            categorical_feature=categorical_features,
            callbacks=callbacks,
        )

        y_pred = model.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        fold_scores.append(acc)

        trial.report(float(np.mean(fold_scores)), step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return float(np.mean(fold_scores))

# --- Optuna設定 ---
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=2)
study = optuna.create_study(direction='maximize', sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\n=== Optuna 結果 ===")
print(f"Best CV accuracy: {study.best_value:.5f}")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

# --- 最終モデル学習 ---
best_params = study.best_params | {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'n_estimators': 600,
    'metric': 'None',
    'random_state': 42,
}
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X_full, y_full, categorical_feature=categorical_features)
print("\n最終モデル学習完了")


[I 2025-10-20 00:38:56,349] A new study created in memory with name: no-name-2181eb6b-5b2b-43eb-a058-7f4802638844
Best trial: 0. Best value: 0.807411:   2%|▎         | 1/40 [00:03<02:12,  3.39s/it]

[I 2025-10-20 00:38:59,736] Trial 0 finished with value: 0.8074111049237243 and parameters: {'learning_rate': 0.02757359293934948, 'num_leaves': 49, 'min_data_in_leaf': 76, 'feature_fraction': 0.8795975452591109, 'bagging_fraction': 0.7468055921327309, 'bagging_freq': 1, 'lambda_l1': 0.001493656855461764, 'lambda_l2': 0.39676050770529864}. Best is trial 0 with value: 0.8074111049237243.


Best trial: 1. Best value: 0.80901:   5%|▌         | 2/40 [00:05<01:45,  2.78s/it] 

[I 2025-10-20 00:39:02,088] Trial 1 finished with value: 0.8090099353175532 and parameters: {'learning_rate': 0.05092911283433821, 'num_leaves': 40, 'min_data_in_leaf': 11, 'feature_fraction': 0.9909729556485983, 'bagging_fraction': 0.9497327922401265, 'bagging_freq': 2, 'lambda_l1': 0.0035113563139704067, 'lambda_l2': 0.0035498788321965025}. Best is trial 1 with value: 0.8090099353175532.


Best trial: 1. Best value: 0.80901:   8%|▊         | 3/40 [00:09<01:53,  3.07s/it]

...

[I 2025-10-20 00:40:04,699] Trial 37 pruned. 


Best trial: 33. Best value: 0.810174:  98%|█████████▊| 39/40 [01:09<00:01,  1.85s/it]

[I 2025-10-20 00:40:06,263] Trial 38 pruned. 


Best trial: 33. Best value: 0.810174: 100%|██████████| 40/40 [01:11<00:00,  1.79s/it]


[I 2025-10-20 00:40:08,044] Trial 39 pruned. 

=== Optuna 結果 ===
Best CV accuracy: 0.81017
learning_rate: 0.028130654811880963
num_leaves: 31
min_data_in_leaf: 49
feature_fraction: 0.9615959375102509
bagging_fraction: 0.9535733043385527
bagging_freq: 4
lambda_l1: 0.34646004336713787
lambda_l2: 0.5988142347842055

最終モデル学習完了


In [18]:
preds = final_model.predict(X_test_full)


passenger_ids = test["PassengerId"]

# DataFrame にまとめる
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Transported": preds
})

# CSV 出力（インデックスは不要）
submission.to_csv("submission_lightgbm.csv", index=False)

submission['Transported'].value_counts()

Transported
True     2142
False    2135
Name: count, dtype: int64

## catboost

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score
import catboost as cb
import optuna
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# =========================
# 1. データ読み込み
# =========================
train = pd.read_csv("../dataset/train.csv")
test = pd.read_csv("../dataset/test.csv")

def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """欠損値を補完せずに特徴量を生成する"""
    data = df.copy()

    # Cabin情報を分解（欠損はNaNのまま保持）
    if 'Cabin' in data.columns:
        cabin_split = data['Cabin'].str.split('/', expand=True)
        data['Deck'] = cabin_split[0]
        data['Num'] = pd.to_numeric(cabin_split[1], errors='coerce')  # NaNそのまま
        data['Side'] = cabin_split[2]
    else:
        data['Deck'] = np.nan
        data['Num'] = np.nan
        data['Side'] = np.nan

    # 支出関連特徴量（NaNはそのまま残す）
    expense_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    two_expense = ['FoodCourt', 'ShoppingMall']
    three_expense = ['RoomService', 'Spa', 'VRDeck']
    for col in expense_cols:
        if col not in data.columns:
            data[col] = np.nan

    data['TotalExpense'] = data[expense_cols].sum(axis=1, skipna=False)
    data['TwoExpense'] = data[two_expense].sum(axis=1, skipna=False)
    data['ThreeExpense'] = data[three_expense].sum(axis=1, skipna=False)
    data['HasExpense'] = data[expense_cols].gt(0).any(axis=1).astype(int)

    # 年齢グループ（NaNはNaNのまま）
    bins = [0, 5, 13, 18, 60, 100]
    labels = ['Child', 'Teen', 'Young', 'Adult', 'Senior']
    data['AgeGroup'] = pd.cut(data['Age'], bins=bins, labels=labels, right=False)

    # Name→家族サイズ
    if 'Name' in data.columns:
        data['FamilyName'] = data['Name'].str.split().str[-1]
        family_counts = data['FamilyName'].value_counts()
        data['FamilySize'] = data['FamilyName'].map(family_counts)
        data['IsAlone'] = (data['FamilySize'] == 1).astype(int)
    else:
        data['FamilyName'] = np.nan
        data['FamilySize'] = np.nan
        data['IsAlone'] = np.nan

    # PassengerId→グループ
    if 'PassengerId' in data.columns:
        passenger_split = data['PassengerId'].str.split('_', expand=True)
        data['PassengerGroup'] = passenger_split[0]
        data['PassengerNum'] = pd.to_numeric(passenger_split[1], errors='coerce')
    else:
        data['PassengerGroup'] = np.nan
        data['PassengerNum'] = np.nan

    return data

# =========================
# 2. 特徴量生成
# =========================
train_features = create_features(train)
test_features = create_features(test)

feature_cols = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'Side', 'Num',
    'TwoExpense', 'ThreeExpense', 'AgeGroup'
]

X_full = train_features[feature_cols].copy()
X_test_full = test_features[feature_cols].copy()

y_full = train['Transported']

# CatBoostに渡すカテゴリ列（列名）
categorical_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'Side', 'AgeGroup'
]

def fix_catboost_cats(df: pd.DataFrame, cat_cols: list[str]) -> pd.DataFrame:
    """CatBoost用にカテゴリ列のNaNを文字列に置換（NaN禁止のため）"""
    df = df.copy()
    for c in cat_cols:
        df[c] = df[c].astype('object')          # object化（bool/category混在OK）
        df[c] = df[c].where(df[c].notna(), 'NaN')  # NaN -> 'NaN'
    return df

X_full = fix_catboost_cats(X_full, categorical_features)
X_test_full = fix_catboost_cats(X_test_full, categorical_features)

# CatBoostは列インデックス指定
cat_features_idx = [feature_cols.index(c) for c in categorical_features]

print(f"使用特徴量数: {len(feature_cols)}")
print(f"特徴量一覧: {feature_cols}")
print(f"データシェイプ: Train {X_full.shape}, Test {X_test_full.shape}")

# =========================
# 3. Optuna 目的関数（CatBoost）
# =========================
def objective(trial: optuna.Trial) -> float:
    params = {
        'loss_function': 'Logloss',
        'eval_metric': 'Logloss',
        'iterations': 600,  # LGBのn_estimatorsに相当
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'depth': 3,  # LGBのmax_depth=3に対応（固定）
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 1.0, log=True),
        'rsm': trial.suggest_float('rsm', 0.7, 1.0),  # 列サブサンプ（feature_fraction相当）
        'bootstrap_type': 'Bernoulli',
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),  # 行サブサンプ（bagging_fraction相当）

        # 追加（重要パラ）
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'one_hot_max_size': trial.suggest_int('one_hot_max_size', 2, 12),

        'random_seed': 42,
        'verbose': False,
        'allow_writing_files': False,
    }

    # PassengerGroupでグループ分割（元コード踏襲）
    groups = train_features['PassengerGroup'].astype(str)
    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

    fold_scores = []
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_full, y_full, groups=groups), start=1):
        X_trn, X_val = X_full.iloc[trn_idx], X_full.iloc[val_idx]
        y_trn, y_val = y_full.iloc[trn_idx], y_full.iloc[val_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            X_trn, y_trn,
            eval_set=(X_val, y_val),
            cat_features=cat_features_idx,
            use_best_model=True,
            early_stopping_rounds=50,
            verbose=False,
        )

        # 予測（クラスラベル）
        y_pred = model.predict(X_val, prediction_type="Class")
        y_pred = np.array(y_pred).ravel().astype(int)

        acc = accuracy_score(y_val, y_pred)
        fold_scores.append(acc)

        trial.report(float(np.mean(fold_scores)), step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return float(np.mean(fold_scores))

# =========================
# 4. Optuna 実行
# =========================
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=2)
study = optuna.create_study(direction='maximize', sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\n=== Optuna 結果 ===")
print(f"Best CV accuracy: {study.best_value:.5f}")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

# =========================
# 5. 最終モデル学習
# =========================
best_params = {
    **study.best_params,
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'iterations': 600,
    'depth': 3,
    'bootstrap_type': 'Bernoulli',
    'random_seed': 42,
    'verbose': False,
    'allow_writing_files': False,
}
final_model = cb.CatBoostClassifier(**best_params)
final_model.fit(
    X_full, y_full,
    cat_features=cat_features_idx,
    verbose=False
)
print("\n最終モデル学習完了")


[I 2025-10-20 00:41:35,691] A new study created in memory with name: no-name-da836c16-ba01-4083-99ec-05eece4bf9e5


使用特徴量数: 9
特徴量一覧: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Side', 'Num', 'TwoExpense', 'ThreeExpense', 'AgeGroup']
データシェイプ: Train (8693, 9), Test (4277, 9)


Best trial: 0. Best value: 0.802362:   2%|▎         | 1/40 [00:06<03:56,  6.07s/it]

[I 2025-10-20 00:41:41,760] Trial 0 finished with value: 0.8023615285731726 and parameters: {'learning_rate': 0.02757359293934948, 'l2_leaf_reg': 0.7114476009343417, 'rsm': 0.9195981825434215, 'subsample': 0.8795975452591109, 'random_strength': 0.15601864044243652, 'one_hot_max_size': 3}. Best is trial 0 with value: 0.8023615285731726.


Best trial: 0. Best value: 0.802362:   5%|▌         | 2/40 [00:08<02:35,  4.08s/it]

...

[I 2025-10-20 00:42:41,865] Trial 37 pruned. 


Best trial: 31. Best value: 0.810674:  98%|█████████▊| 39/40 [01:07<00:01,  1.34s/it]

[I 2025-10-20 00:42:43,420] Trial 38 finished with value: 0.8089459100087287 and parameters: {'learning_rate': 0.10102941245139367, 'l2_leaf_reg': 0.030621781901493637, 'rsm': 0.959730278205339, 'subsample': 0.9454306910677724, 'random_strength': 0.405317946886742, 'one_hot_max_size': 7}. Best is trial 31 with value: 0.8106736789042326.


Best trial: 31. Best value: 0.810674: 100%|██████████| 40/40 [01:08<00:00,  1.71s/it]


[I 2025-10-20 00:42:44,281] Trial 39 pruned. 

=== Optuna 結果 ===
Best CV accuracy: 0.81067
learning_rate: 0.1469444268239764
l2_leaf_reg: 0.029549134596372463
rsm: 0.8251054272088577
subsample: 0.9985683049575292
random_strength: 0.8939262780900099
one_hot_max_size: 10

最終モデル学習完了


In [20]:
preds = final_model.predict(X_test_full)


passenger_ids = test["PassengerId"]

# DataFrame にまとめる
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Transported": preds
})

# CSV 出力（インデックスは不要）
submission.to_csv("submission_catboost.csv", index=False)

submission['Transported'].value_counts()

Transported
True     2179
False    2098
Name: count, dtype: int64

## catboost byさわぐち

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import accuracy_score
import catboost as cb
import optuna
import warnings

warnings.filterwarnings('ignore', category=UserWarning)

# =========================
# 1. データ読み込み
# =========================
train = pd.read_csv("../dataset/train.csv")
test = pd.read_csv("../dataset/test.csv")

def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """欠損値を補完せずに特徴量を生成する"""
    data = df.copy()

    # Cabin情報を分解（欠損はNaNのまま保持）
    if 'Cabin' in data.columns:
        cabin_split = data['Cabin'].str.split('/', expand=True)
        data['Deck'] = cabin_split[0]
        data['Num'] = pd.to_numeric(cabin_split[1], errors='coerce')  # NaNそのまま
        data['Side'] = cabin_split[2]
    else:
        data['Deck'] = np.nan
        data['Num'] = np.nan
        data['Side'] = np.nan

    # 支出関連特徴量（NaNはそのまま残す）
    expense_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    two_expense = ['FoodCourt', 'ShoppingMall']
    three_expense = ['RoomService', 'Spa', 'VRDeck']
    for col in expense_cols:
        if col not in data.columns:
            data[col] = np.nan

    data['TotalExpense'] = data[expense_cols].sum(axis=1, skipna=False)
    data['TwoExpense'] = data[two_expense].sum(axis=1, skipna=False)
    data['ThreeExpense'] = data[three_expense].sum(axis=1, skipna=False)
    data['HasExpense'] = data[expense_cols].gt(0).any(axis=1).astype(int)

    # 年齢グループ（NaNはNaNのまま）
    bins = [0, 5, 13, 18, 60, 100]
    labels = ['Child', 'Teen', 'Young', 'Adult', 'Senior']
    data['AgeGroup'] = pd.cut(data['Age'], bins=bins, labels=labels, right=False)

  # Name情報から家族サイズを生成（欠損はNaN）
    if 'Name' in data.columns:
        data['FamilyName'] = data['Name'].str.split().str[-1]
        family_counts = data['FamilyName'].value_counts()
        data['FamilySize'] = data['FamilyName'].map(family_counts)
        data['IsAlone'] = (data['FamilySize'] == 1).astype(int)
    # PassengerId情報（欠損時はNaN）
    if 'PassengerId' in data.columns:
        passenger_split = data['PassengerId'].str.split('_', expand=True)
        data['PassengerGroup'] = passenger_split[0]
        data['PassengerNum'] = pd.to_numeric(passenger_split[1], errors='coerce')
        
    # 組み合わせ特徴量の追加
    num_bins = [0, 250, 500, 750, 2000]
    num_labels = ['Q1', 'Q2', 'Q3', 'Q4']
    data['NumQuartile'] = pd.cut(data['Num'], bins=num_bins, labels=num_labels, right=False)
    data['Cabin_Loc'] = data['Deck'].astype(str) + '_' + data['NumQuartile'].astype(str)
    data['NoExpense_Cryo'] = ((data['CryoSleep'] == True) & (data['TotalExpense'] == 0)).astype(object)
    data['Cryo_VIP'] = data['CryoSleep'].astype(str) + '_' + data['VIP'].astype(str)
    data['Planet_AgeGroup'] = data['HomePlanet'].astype(str) + '_' + data['AgeGroup'].astype(str)
    data['Deck_Side'] = data['Deck'].astype(str) + '_' + data['Side'].astype(str)
    data['Full_Travel_Combo'] = (data['HomePlanet'].astype(str) + '_' + 
                                 data['Destination'].astype(str) + '_' + 
                                 data['CryoSleep'].astype(str))
    if 'FamilyName' in data.columns:
        data['Family_Group'] = data['FamilyName'].astype(str) + '_' + data['PassengerGroup'].astype(str)
    else:
        data['Family_Group'] = np.nan
    return data
    
# =========================================================
# 2. 特徴量生成とデータ準備
# =========================================================
train_features = create_features(train)
test_features = create_features(test)

feature_cols = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'Side', 'Num',
    'TwoExpense', 'ThreeExpense', 'AgeGroup',
    'Cabin_Loc', 'NoExpense_Cryo', 'Cryo_VIP', 'Planet_AgeGroup',
    'Deck_Side', 'Full_Travel_Combo',
]

X_full = train_features[feature_cols].copy()
X_test_full = test_features[feature_cols].copy()
y_full = train['Transported']

categorical_features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'VIP',
    'Side', 'AgeGroup',
    'Cabin_Loc', 'NoExpense_Cryo', 'Cryo_VIP', 'Planet_AgeGroup',
    'Deck_Side', 'Full_Travel_Combo',
]

for col in categorical_features:
    X_full[col] = X_full[col].astype(str).replace('nan', 'NA').replace('None', 'NA')
    X_test_full[col] = X_test_full[col].astype(str).replace('nan', 'NA').replace('None', 'NA')

groups = train_features['PassengerGroup'].fillna('Unknown').astype(str)

print(f"使用特徴量数: {len(feature_cols)}")
print(f"特徴量一覧: {feature_cols}")
print(f"データシェイプ: Train {X_full.shape}, Test {X_test_full.shape}")


def fix_catboost_cats(df: pd.DataFrame, cat_cols: list[str]) -> pd.DataFrame:
    """CatBoost用にカテゴリ列のNaNを文字列に置換（NaN禁止のため）"""
    df = df.copy()
    for c in cat_cols:
        df[c] = df[c].astype('object')          # object化（bool/category混在OK）
        df[c] = df[c].where(df[c].notna(), 'NaN')  # NaN -> 'NaN'
    return df

X_full = fix_catboost_cats(X_full, categorical_features)
X_test_full = fix_catboost_cats(X_test_full, categorical_features)

# CatBoostは列インデックス指定
cat_features_idx = [feature_cols.index(c) for c in categorical_features]

print(f"使用特徴量数: {len(feature_cols)}")
print(f"特徴量一覧: {feature_cols}")
print(f"データシェイプ: Train {X_full.shape}, Test {X_test_full.shape}")

# =========================
# 3. Optuna 目的関数（CatBoost）
# =========================
def objective(trial: optuna.Trial) -> float:
    params = {
        'loss_function': 'Logloss',
        'eval_metric': 'Logloss',
        'iterations': 600,  # LGBのn_estimatorsに相当
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
        'depth': 3,  # LGBのmax_depth=3に対応（固定）
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 1.0, log=True),
        'rsm': trial.suggest_float('rsm', 0.7, 1.0),  # 列サブサンプ（feature_fraction相当）
        'bootstrap_type': 'Bernoulli',
        'subsample': trial.suggest_float('subsample', 0.7, 1.0),  # 行サブサンプ（bagging_fraction相当）

        # 追加（重要パラ）
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'one_hot_max_size': trial.suggest_int('one_hot_max_size', 2, 12),

        'random_seed': 42,
        'verbose': False,
        'allow_writing_files': False,
    }

    # PassengerGroupでグループ分割（元コード踏襲）
    groups = train_features['PassengerGroup'].astype(str)
    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)

    fold_scores = []
    for fold_idx, (trn_idx, val_idx) in enumerate(cv.split(X_full, y_full, groups=groups), start=1):
        X_trn, X_val = X_full.iloc[trn_idx], X_full.iloc[val_idx]
        y_trn, y_val = y_full.iloc[trn_idx], y_full.iloc[val_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            X_trn, y_trn,
            eval_set=(X_val, y_val),
            cat_features=cat_features_idx,
            use_best_model=True,
            early_stopping_rounds=50,
            verbose=False,
        )

        # 予測（クラスラベル）
        y_pred = model.predict(X_val, prediction_type="Class")
        y_pred = np.array(y_pred).ravel().astype(int)

        acc = accuracy_score(y_val, y_pred)
        fold_scores.append(acc)

        trial.report(float(np.mean(fold_scores)), step=fold_idx)
        if trial.should_prune():
            raise optuna.TrialPruned()

    return float(np.mean(fold_scores))

# =========================
# 4. Optuna 実行
# =========================
sampler = optuna.samplers.TPESampler(seed=42)
pruner = optuna.pruners.MedianPruner(n_warmup_steps=2)
study = optuna.create_study(direction='maximize', sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("\n=== Optuna 結果 ===")
print(f"Best CV accuracy: {study.best_value:.5f}")
for k, v in study.best_params.items():
    print(f"{k}: {v}")

# =========================
# 5. 最終モデル学習
# =========================
best_params = {
    **study.best_params,
    'loss_function': 'Logloss',
    'eval_metric': 'Logloss',
    'iterations': 600,
    'depth': 3,
    'bootstrap_type': 'Bernoulli',
    'random_seed': 42,
    'verbose': False,
    'allow_writing_files': False,
}
final_model = cb.CatBoostClassifier(**best_params)
final_model.fit(
    X_full, y_full,
    cat_features=cat_features_idx,
    verbose=False
)
print("\n最終モデル学習完了")


[I 2025-10-20 00:44:40,429] A new study created in memory with name: no-name-9eb55eee-e1e2-4fd8-9d99-6dadce7be032


使用特徴量数: 15
特徴量一覧: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Side', 'Num', 'TwoExpense', 'ThreeExpense', 'AgeGroup', 'Cabin_Loc', 'NoExpense_Cryo', 'Cryo_VIP', 'Planet_AgeGroup', 'Deck_Side', 'Full_Travel_Combo']
データシェイプ: Train (8693, 15), Test (4277, 15)
使用特徴量数: 15
特徴量一覧: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Side', 'Num', 'TwoExpense', 'ThreeExpense', 'AgeGroup', 'Cabin_Loc', 'NoExpense_Cryo', 'Cryo_VIP', 'Planet_AgeGroup', 'Deck_Side', 'Full_Travel_Combo']
データシェイプ: Train (8693, 15), Test (4277, 15)


Best trial: 0. Best value: 0.813957:   2%|▎         | 1/40 [00:07<04:42,  7.24s/it]

[I 2025-10-20 00:44:47,665] Trial 0 finished with value: 0.8139566998237306 and parameters: {'learning_rate': 0.02757359293934948, 'l2_leaf_reg': 0.7114476009343417, 'rsm': 0.9195981825434215, 'subsample': 0.8795975452591109, 'random_strength': 0.15601864044243652, 'one_hot_max_size': 3}. Best is trial 0 with value: 0.8139566998237306.


Best trial: 0. Best value: 0.813957:   5%|▌         | 2/40 [00:12<03:59,  6.31s/it]

...

[I 2025-10-20 00:47:02,882] Trial 37 pruned. 


Best trial: 29. Best value: 0.819072:  98%|█████████▊| 39/40 [02:25<00:03,  3.03s/it]

[I 2025-10-20 00:47:06,225] Trial 38 pruned. 


Best trial: 29. Best value: 0.819072: 100%|██████████| 40/40 [02:31<00:00,  3.78s/it]


[I 2025-10-20 00:47:11,719] Trial 39 finished with value: 0.8181557068877201 and parameters: {'learning_rate': 0.07033006166215403, 'l2_leaf_reg': 0.04816033445210995, 'rsm': 0.830020688510257, 'subsample': 0.836266505470355, 'random_strength': 0.6198649120051961, 'one_hot_max_size': 9}. Best is trial 29 with value: 0.8190722393014198.

=== Optuna 結果 ===
Best CV accuracy: 0.81907
learning_rate: 0.05226171882074134
l2_leaf_reg: 0.9826740452177546
rsm: 0.8193592936801084
subsample: 0.8732774399616086
random_strength: 0.5471401318275659
one_hot_max_size: 11

最終モデル学習完了


In [22]:
preds = final_model.predict(X_test_full)


passenger_ids = test["PassengerId"]

# DataFrame にまとめる
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Transported": preds
})

# CSV 出力（インデックスは不要）
submission.to_csv("submission_catboost_many_features.csv", index=False)

submission['Transported'].value_counts()

Transported
True     2192
False    2085
Name: count, dtype: int64